#Data scraping and preprocessing

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re

In [ ]:
driver = webdriver.Chrome('add your local path to chromewebdriver.exe')
driver.get('https://www.simplyscripts.com/genre/romance-scripts.html')
time.sleep(5)
doc = driver.page_source
counter = 0
counter1 = 0
soup = BeautifulSoup(doc, features='html.parser')

In [ ]:
for link in soup.find_all('a'):
    counter += 1
    lnk = link.get('href')
    # Check whether the link has a text file, returns a 200 code and is not of a defunct website
    if(lnk.endswith('txt') and requests.get(lnk).status_code == 200 and 'weeklyscript.com' not in lnk and 'angelfire.com' not in lnk):        
        counter1 += 1
        driver.get(lnk)
        doc1 = driver.page_source
        soup1 = BeautifulSoup(doc1, features='html.parser')
        with open('traindata1.txt', 'a') as f:
            for txt in soup1.find_all('pre'):
                pattern = r'<pre style="word-wrap: break-word; white-space: pre-wrap;">'
                txt = re.sub(pattern, '', str(txt))
                pattern = r'</pre>'
                txt = re.sub(pattern, '', str(txt))
                f.write(txt)
    else:
        print(str(counter) + ". No text files found!")

print("Total scripts saved:", counter1)

# Markov Model (some code is courtesy of @ashwinmj)

In [ ]:
import string
import numpy as np

In [ ]:
# Path of the text file containing the training data
data = 'traindata1.txt'

In [ ]:
#Helper functions
def remPunc(sentence):
    return sentence.translate(str.maketrans('','', string.punctuation))

def dictAdd(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = []
    dictionary[key].append(value)

def probDictAdd(lst):
    probs = {}
    lstLen = len(lst)
    for item in lst:
        probs[item] = probs.get(item, 0) + 1
    for key, value in probs.items():
        probs[key] = value / lstLen
    return probs

def randomWord(dictionary):
    p0 = np.random.random()
    cumulative = 0
    for key, value in dictionary.items():
        cumulative += value
        if p0 < cumulative:
            return key
    assert(False)

firstWord = {}
secondWord = {}
transitions = {}

In [ ]:
# Trains a Markov model based on the data in data
def train():
    for line in open(data, encoding='latin-1'):
        tokens = remPunc(line.rstrip().lower()).split()
        tokensLen = len(tokens)
        for i in range(tokensLen):
            token = tokens[i]
            if i == 0:
                firstWord[token] = firstWord.get(token, 0) + 1
            else:
                prev = tokens[i - 1]
                if i == tokensLen - 1:
                    dictAdd(transitions, (prev, token), 'END')
                if i == 1:
                    dictAdd(secondWord, prev, token)
                else:
                    prevPrev = tokens[i - 2]
                    dictAdd(transitions, (prevPrev, prev), token)
    
    # Normalize the distributions
    total = sum(firstWord.values())
    for key, value in firstWord.items():
        firstWord[key] = value / total
        
    for prevWord, nextWordLst in secondWord.items():
        secondWord[prevWord] = probDictAdd(nextWordLst)
        
    for pair, nextWordLst in transitions.items():
        transitions[pair] = probDictAdd(nextWordLst)
    
    print('Training successful.')

In [ ]:
numPred = 15

# Function to autocomplete sample text
def autocomplete(w0, w1):
    for i in range(numPred):
        sentence = []
        # Initial word
        word0 = w0
        sentence.append(word0)
        # Second word
        word1 = w1
        sentence.append(word1)
        # Subsequent words untill END
        endc = 0
        while True:
            if word1 == 'END':
              break
            word2 = randomWord(transitions[(word0, word1)])
            while word2 == 'END':
                endc+=1
                if endc >= 100:
                  break
                else:
                  word2 = randomWord(transitions[(word0, word1)])
            sentence.append(word2)
            word0 = word1
            word1 = word2
        print(' '.join(sentence))

In [ ]:
np.random.seed(0)
train()

Training successful.


In [ ]:
np.random.seed(10)
autocomplete('she', 'feels')

she feels for him for six years ago i barely made it up looks briefly to his door leaving melvin alone in the rear windows exploding outward in a shop being closed other shops END
she feels like shes some fantastic new species END
she feels a presence at her clothes of all these houses were convents END
she feels she takes off the plate of eggs bacon and sausage with END
she feels kind of wild huh END
she feels like how being in the small size i always say no one helps them when they break apart they stretch themselves corrado END
she feels she takes it aboard the marshal starts climbing up towards the jury waits anxiously for her to abort her mission completed she turns and takes him a deep breath he turns to snow where the wavin wheat can sure smell END
she feels kind of pers END
she feels like how being in love with architecture END
she feels separate from him the door opens and the first time its perry the professor coming END
she feels kind of person END
she feels kind of roar fro

#Recurrent Neural Network Model v1



In [ ]:
import os 
import tensorflow as tf
tf.compat.v1.enable_eager_execution()


In [ ]:
try:
  device_name = os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS = 'grpc://' + device_name
  print('Found TPU at: {}'.format(TPU_ADDRESS))
  cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_ADDRESS)
  tf.config.experimental_connect_to_cluster(cluster_resolver)
  tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
  tpu_strategy = tf.distribute.TPUStrategy(cluster_resolver)
except KeyError:
 print('TPU not found')

Found TPU at: grpc://10.85.8.10:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.85.8.10:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.85.8.10:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [ ]:
# load ascii text and covert to lowercase
filename = "traindata1.txt"
raw_text = open(filename, 'r', encoding='latin-1').read()
raw_text = raw_text.lower()

# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

Total Characters:  2456084
Total Vocab:  77
Total Patterns:  2455984


In [ ]:
with tpu_strategy.scope():
  # define the LSTM model
  model = Sequential()
  model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
  model.add(Dropout(0.1))
  model.add(LSTM(128))
  model.add(Dropout(0.1))
  model.add(Dense(y.shape[1], activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=20, batch_size=64, callbacks=callbacks_list)

Epoch 1/20
38375/38375 [==============================] - 714s 18ms/step - loss: 2.4581

Epoch 00001: loss improved from inf to 2.27012, saving model to weights-improvement-01-2.2701.hdf5
Epoch 2/20
38375/38375 [==============================] - 706s 18ms/step - loss: 1.9910

Epoch 00002: loss improved from 2.27012 to 1.94571, saving model to weights-improvement-02-1.9457.hdf5
Epoch 3/20
38375/38375 [==============================] - 699s 18ms/step - loss: 1.8397

Epoch 00003: loss improved from 1.94571 to 1.81851, saving model to weights-improvement-03-1.8185.hdf5
Epoch 4/20
38375/38375 [==============================] - 701s 18ms/step - loss: 1.7598

Epoch 00004: loss improved from 1.81851 to 1.74710, saving model to weights-improvement-04-1.7471.hdf5
Epoch 5/20
38375/38375 [==============================] - 702s 18ms/step - loss: 1.7053

Epoch 00005: loss improved from 1.74710 to 1.69879, saving model to weights-improvement-05-1.6988.hdf5
Epoch 6/20
38375/38375 [====================

In [ ]:
# load the network weights
filename = "weights-improvement-20-1.4735.hdf5"
model.load_weights(filename)
with tpu_strategy.scope():
  model.compile(loss='categorical_crossentropy', optimizer='adam')

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" n the open.  a moment later, dyle 
	passes a girl painting a canvas, her easel set up in the
	middle "
 of the shoulders and she sees the shoulder and the
	sears of the shoulders and she sees the shoulde
Done.


#Recurrent Neural Network v2

In [ ]:
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
from keras.utils import to_categorical

training_doc3 = open('traindata1.txt', encoding='latin-1').read()
training_doc3 = training_doc3[:100000]
cleaned = re.sub(r'\W+', ' ', training_doc3).lower()
tokens = word_tokenize(cleaned)
train_len = 4
text_sequences = []

for i in range(train_len,len(tokens)):
  seq = tokens[i-train_len:i]
  text_sequences.append(seq)

sequences = {}
count = 1

for i in range(len(tokens)):
  if tokens[i] not in sequences:
    sequences[tokens[i]] = count
    count += 1

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)

#vocabulary size increased by 1 for the cause of padding
vocabulary_size = len(tokenizer.word_counts)+1
n_sequences = np.empty([len(sequences),train_len], dtype='int32')

for i in range(len(sequences)):
  n_sequences[i] = sequences[i]

train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

import os 
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

try:
  device_name = os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS = 'grpc://' + device_name
  print('Found TPU at: {}'.format(TPU_ADDRESS))
  cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_ADDRESS)
  tf.config.experimental_connect_to_cluster(cluster_resolver)
  tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
  tpu_strategy = tf.distribute.TPUStrategy(cluster_resolver)
except KeyError:
 print('TPU not found')

Found TPU at: grpc://10.85.8.10:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.85.8.10:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.85.8.10:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
with tpu_strategy.scope():
  model = Sequential()
  model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
  model.add(LSTM(50,return_sequences=True))
  model.add(LSTM(50))
  model.add(Dense(50,activation='relu'))
  model.add(Dense(vocabulary_size, activation='softmax'))
  # compiling the network
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  

In [ ]:
with tpu_strategy.scope():
  model.fit(train_inputs,train_targets,epochs=500,verbose=1)  

Epoch 1/500
553/553 [==============================] - 9s 17ms/step - loss: 3.4603 - accuracy: 0.2346
Epoch 2/500
553/553 [==============================] - 9s 16ms/step - loss: 3.4397 - accuracy: 0.2394
Epoch 3/500
553/553 [==============================] - 9s 17ms/step - loss: 3.4133 - accuracy: 0.2419
Epoch 4/500
553/553 [==============================] - 9s 17ms/step - loss: 3.3894 - accuracy: 0.2461
Epoch 5/500
553/553 [==============================] - 9s 16ms/step - loss: 3.3715 - accuracy: 0.2495
Epoch 6/500
553/553 [==============================] - 9s 16ms/step - loss: 3.3501 - accuracy: 0.2512
Epoch 7/500
553/553 [==============================] - 9s 16ms/step - loss: 3.3272 - accuracy: 0.2557
Epoch 8/500
553/553 [==============================] - 9s 16ms/step - loss: 3.3100 - accuracy: 0.2611
Epoch 9/500
553/553 [==============================] - 9s 16ms/step - loss: 3.2858 - accuracy: 0.2667
Epoch 10/500
553/553 [==============================] - 9s 16ms/step - loss: 3.268

In [ ]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(encoded_text, pad_encoded)
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

hello how are
[941, 125, 29] [[941 125  29]]


ValueError: ignored